# Исследование надежности заемщиков.

В ходе данной работы рассмотрим и сравним долю невозвратов/просрочек кредитов среди различных групп заёмщиков и в зависимости от целей кредита. Для исследования возьмем статистику банка по фактическим заёмщикам, их характеристикам и фактам невозвратов/просрочек кредитов и обработаем её для дальнейшего изучения. Полученные выводу помогут определить наиболее и наименее ответственные группы заёмщиков в зависимости от их семейного положения, социального статуса, цели кредита и прочего.

### Шаг 1. Обзор данных

Окрываем файл и просматриваем первые 10 строк таблицы для получения представления его структуре и формате данных

In [1]:
import pandas as pd
df=pd.read_csv('/datasets/data.csv')
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


А также общую информацию о размере датафрейма и типе данных

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Дата фрейм включает в себя 12 столбцов и 21525 строк различных форматов. Очевидно, что в нём присутствуют пропуски и аномалии. Далее мы от них избавимся. 

### Шаг 2.1 Заполнение пропусков

Определяем столбцы, в которых есть пропуски и их количество

In [3]:
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Пропуски обнаружены в столбцах 'days_employed' - трудовой стаж в днях и 'total_income' - суммарный доход.


**Заполнение пропусков стажа**<br>Пропуски в 'days_employed' целесообразно заполнять, исходя из соотношения среднего возраста людей к среднему или медианному трудовому стажу.
Сделаем проверку - сравнение среднего и медианного стажа в пересчёте на годы для определения того, какой стаж выбрать.

In [4]:
print('Средний стаж',df['days_employed'].mean()/365)
print('Медианный стаж',df['days_employed'].median()/365)

Средний стаж 172.73013057937914
Медианный стаж -3.296902818549285


Очевидно, медианный стаж имеет аномалии, вызванные значениями, введёнными со знаком "-", а средний стаж - аномалии, связанные с неправдоподобно большими значениями. Попробуем взять медиану по абсолютному значению стажа: 

In [5]:
print('Медианный стаж',abs(df['days_employed']).median()/365)

Медианный стаж 6.0115631969279315


Медианный стаж выглядит достаточно небольшим, но необходимо проверить средний возраст для сопоставления со стажем, т.к. выборка может состоять из относительно молодых людей, для которых 6 лет трудового стажа будут адекватной величиной.

In [6]:
print('Средний возраст',df['dob_years'].mean())

Средний возраст 43.29337979094077


Очевидно, что для среднего возраста 43 года стаж в 6 лет является неадекватно маленьким. Попробуем другую теорию: большие значения стажа - это стаж в часах. Предельным стажем будем считать 50 лет, все значения, превышающие этот показатель будут аномалиями. Проверим количество строк для 50 лет, а также, для проверки, явно аномальные значения со стажем более 100000 и 300000 дней: 

In [7]:
print('Люди со стажем более 50 лет:',df[df['days_employed']>(50*365)]['days_employed'].count())
print('Люди со стажем более 100000 "дней":',df[df['days_employed']>100000]['days_employed'].count())
print('Люди со стажем более 300000 "дней":',df[df['days_employed']>100000]['days_employed'].count())

Люди со стажем более 50 лет: 3445
Люди со стажем более 100000 "дней": 3445
Люди со стажем более 300000 "дней": 3445


Таким образом, все данные о стаже, превышающем 50 лет будем считать данными, выраженными в часах, и конвертировать в дни.
<a id='hours_to_years'></a>

In [8]:
df.loc[df['days_employed']>(50*365),'days_employed']=df['days_employed']/24

Теперь проверим средний и медианный стаж в годах с учётом исправлений и абсолютных значений:

In [9]:
print('Медианный стаж',abs(df['days_employed']).median()/365)
print('Средний стаж',abs(df['days_employed']).mean()/365)

Медианный стаж 6.0115631969279315
Средний стаж 12.716825140220974


Очевидно, средний стаж в 12,7 лет соответствует среднему возрасту заемщика в 43 года. Теперь можно посчитать среднее значение отношения стажа к возрасту и заполнить пропуски трудового стажа заемщика, исходя из его возраста:

In [10]:
avg_age_empl_ratio=abs(df['days_employed']).mean()/df['dob_years'].mean()
#df.loc[df['days_employed']=='None','days_employed']=df['dob_years']*avg_age_empl_ratio
df['days_employed']=df['days_employed'].fillna(df['dob_years']*avg_age_empl_ratio)
print(df.isna().sum())

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


**Заполнение пропусков среднего заработка**<br>Теперь произведём заполнение пропусков среднего заработка. Предположение такое, что люди, имеющие одинаковое образование, имеют сопоставимый доход. Выведем все уникальные значения по образованию, имеющиеся в датафрейме.

In [11]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Унифицируем все значения образования - приведём их в форму строчных букв и посмотрим на уникальные значения.
<a id='lower'></a>

In [12]:
df['education']=df['education'].str.lower()
df['education'].unique()


array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Объявим список, состоящий из уникальных значений типов образования и пустой список средних зарплат в зависимости от образования. Посчитаем медианные зарплаты для каждого типа образования и добавим их в список и выведем на экран.

In [13]:
educ=['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень']
income_to_educ=[]
for index in educ:
    income_to_educ.append(df[df['education']==index]['total_income'].median())
print(educ,'\n',income_to_educ) 

['высшее', 'среднее', 'неоконченное высшее', 'начальное', 'ученая степень'] 
 [175340.81885544234, 136478.64324360332, 160115.39864359115, 117137.35282495996, 157259.8985551767]


Заменим пропуски соответствующими медианными значениями

In [14]:
for i in range(len(educ)):
    df.loc[df['education']==educ[i],'total_income']=df['total_income'].fillna(income_to_educ[i])
print(df.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Шаг 2.2 Проверка данных на аномалии и исправления.

На [предыдущих шагах](#hours_to_years) мы уже заменили аномально большие значения трудового стажа, выраженные в часах вместо дней. Но для расчётов использовали функцию абсолютных значений, без замены отрицательных значений в самом датафрейме. Сделаем все значения стажа положительными на этом этапе:

In [15]:
df['days_employed']=abs(df['days_employed'])

Проверим дополнительно аномалии пола:

In [16]:
print(df['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


В столбцах пола помимо F и M представлен XNA - очевидно, "not available". Но, т.к. в дальнейшем мы не делаем отдельную выборку по половому признаку, а также ввиду того, что это всего одна строка с таким значением, оставим данный столбец неизменным.

Проверим на аномалии также столбец с возрастом заёмщиков.

In [17]:
print(df['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


В датафрейме есть 101 строка с возрастом 0 лет. Очевидно это вызвано ошибкой заполнения. Для целей нашего анализа возраст можно заменить средним значением из имеющихся.

In [18]:
df.loc[df['dob_years']==0,'dob_years']=int(df['dob_years'].mean())
print(df['dob_years'].value_counts())

35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


### Шаг 2.3. Изменение типов данных.

Поменяем тип данных столбца 'total_income' с float на int

In [19]:
df['total_income']=df['total_income'].astype('int')

### Шаг 2.4. Удаление дубликатов.

Определим количество дубликатов и уберем их. Выборка будет включать как явные, так и неявыне дубликаты, т.к. [ранее](#lower) мы уже привели все строки с образованием ('education') к нижнему регистру

In [20]:
print('Обнаружено и удалено дубликатов:', df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop=True)

Обнаружено и удалено дубликатов: 71


Выведем на экран первые 10 строк датафрейма и информацию о датафрейме для проверки

In [21]:
display(df.head(10))
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  int64  
 11  purpose           21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Сформируем два дополнительных датафрейма-словаря, в которых будут храниться данные об образовании и семейном положении с соответствующими идентификаторами. В словарях оставим только уникальные значения с их идентификаторами. Соответственно, удалим столбцы 'education' и 'family_status' из исходного датафрейма, оставив только их идентификаторы.

In [22]:
educ_log=df[['education_id','education']]
family_log=df[['family_status_id','family_status']]
educ_log=educ_log.drop_duplicates().reset_index(drop=True)
family_log=family_log.drop_duplicates().reset_index(drop=True)
df=df.drop(columns=['education','family_status'],axis=1)
display(educ_log.head())
display(family_log.head())
df.columns

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

### Шаг 2.6. Категоризация дохода.

Создадим функцию категоризации заемщиков по уровню их дохода и добавим к основному датафрейму столбец с категориями заемщиков.

In [23]:
def income_cat(total_income):
    if total_income<=30000:
        return 'E'
    if 30001<=total_income<=50000:
        return 'D'
    if 50001<=total_income<=200000:
        return 'C'
    if 200001<=total_income<=1000000:
        return 'B'
    if total_income>=1000001:
        return 'A'
df['total_income_category']=df['total_income'].apply(income_cat)
display(df.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

Аналогично создадим функцию категоризации цели займа. Для этого, предварительно, определим все уникальные значения в данном столбце:


In [24]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Исходя из перечня уникальных значений, для категоризации целей необходимо, чтобы они содержали следуюшие значения:
<br> 'операции с автомобилем' - 'авто'
<br> 'операции с недвижимостью' - 'недв', 'жил'
<br> 'проведение свадьбы' - 'свадьб'
<br> 'получение образования' - 'образ'

Напишем соответствующую функцию категоризации и добавим столбец в датафрейм

In [25]:
def purpose_cat(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'недв' in purpose or 'жил' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образ' in purpose:
        return 'получение образования'
df['purpose_category']=df['purpose'].apply(purpose_cat)
display(df.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.

##### Вопрос 1:Есть ли зависимость между количеством детей и возвратом кредита в срок?

Определим максимальное количество детей среди заёмщиков:

In [26]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Очевидно, в данных о количестве детей также есть аномалии - присутствуют отрицательные значения и неправдоподобно большшое число - 20. Возьмем значения по модулю, а значение 20 заменим на 2, т.к. это вероятнеее всего опечатка

In [27]:
df['children']=abs(df['children'])
df.loc[df['children']==20,'children']=2
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [28]:
def children_debt(row):
    children = row['children']
    debt = row['debt']
    if debt ==1:
        if children == 0:
            return 'без детей'
        if children == 1:
            return '1 ребенок'
        if children == 2:
            return '2 ребенка'
        if children == 3:
            return '3 ребенка'
        if children == 4:
            return '4 ребенка'
        if children == 5:
            return '5 детей'
df['children_debt'] = df.apply(children_debt, axis=1)
print(df['children_debt'].value_counts()) 
print(df['children'].value_counts())


без детей    1063
1 ребенок     445
2 ребенка     202
3 ребенка      27
4 ребенка       4
Name: children_debt, dtype: int64
0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


Сведем итоговый датафрейм и посчитаем долю просроченных кредитов по категориям.

In [29]:


# через pivot
table_children=df.pivot_table(index='children', values='debt',aggfunc=['count','sum'])
table_children['debt share']=table_children['sum']/table_children['count']
table_children.columns=['всего взяли кредит','из них просрочено', 'доля просроченных']
display(table_children)

,всего взяли кредит,из них просрочено,доля просроченных
children,,,
0,14091,1063,0.075438
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


In [30]:
# 

df.groupby('children')['debt'].agg(['count','sum','mean'])

,count,sum,mean
children,,,
0,14091,1063,0.075438
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


In [31]:
# 

def my_mean(x): return x.mean()*100

# В кач-ве агрегирования можно любую функцию брать.
df.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
0,14091,1063,7.543822
1,4855,445,9.165808
2,2128,202,9.492481
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


In [32]:
# 

def my_mean(x): return '{:.2%} '.format(x.mean())

df.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
0,14091,1063,7.54%
1,4855,445,9.17%
2,2128,202,9.49%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%


##### Вывод 1:

Таким образом, получается, что наиболее отвественными (7,5% просроченных кредитов) являются заёмщики без детей, они же и представляют собой самую многочисленную группу заёмщиков. Наименее ответсвенными и допустивщими больше всего просрочек - многодетные заёмщики, имеющие 4 ребенка (9,8% просроченных платежей). Доля задолженности среди заемщиков, имеющих 1-2 ребенка примерно одинакова, но значительно выше, чем у заемщиков без детей. <br>Также, минимальная доля просроченных кредитов среди заёмщиков с детьми (3 ребёнка) значительно превышает аналогичный показатель для заёмщиков без детей. <br>Таким образом, можно говорить о тенденции, что наличие детей отрицательно влияет на возможность возврата кредита в срок

##### Вопрос 2:Есть ли зависимость между семейным положением и возвратом кредита в срок?

Определим количество просроченных кредитов в зависимости от семейного положения:

In [33]:
def family_debt(row):
    family = row['family_status_id']
    debt = row['debt']
    if debt ==1:
        if family == 0:
            return 'женат / замужем'
        if family == 1:
            return 'гражданский брак'
        if family == 2:
            return 'вдовец / вдова'
        if family == 3:
            return 'в разводе'
        if family == 4:
            return 'Не женат / не замужем'
df['family_debt'] = df.apply(family_debt, axis=1)
print(df['family_debt'].value_counts()) 
print(df['family_status_id'].value_counts())

женат / замужем          931
гражданский брак         388
Не женат / не замужем    274
в разводе                 85
вдовец / вдова            63
Name: family_debt, dtype: int64
0    12339
1     4151
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64


Сведем итоговый датафрейм и посчитаем долю просроченных кредитов по категориям.

In [34]:


# через pivot
table_family=df.pivot_table(index='family_status_id', values='debt',aggfunc=['count','sum'])
table_family['debt share']=table_family['sum']/table_family['count']
table_family.columns=['всего взяли кредит','из них просрочено', 'доля просроченных']
table_family1=family_log.join(table_family) #объединим со словарем
table_family1=table_family1.drop(columns=['family_status_id'],axis=1) #уберем колонку с id
display(table_family1)

,family_status,всего взяли кредит,из них просрочено,доля просроченных
0,женат / замужем,12339,931,0.075452
1,гражданский брак,4151,388,0.093471
2,вдовец / вдова,959,63,0.065693
3,в разводе,1195,85,0.071130
4,Не женат / не замужем,2810,274,0.097509


##### Вывод 2:
Наименее отвественными являются холостые заемщики и заемщики, проживающие в граданском браке (9,7% и и 9,3% просроченных платежей, соответственно). Среди остальных групп заемщиков, которе включают в себя женатых в настоящий момент, либо женатых в прошлом, доля просроченных кредитов значительно ниже. Безусловно, этот показатель также должен коррелировать с возрастом и уровнем и стабильностью дохода - женатые старше и имеют больший стабильный доход. Но по результатам данного сравнения неженатые/незамужние клиенты являются менее надежными заёмщиками.

##### Вопрос 3:Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Определим количество просроченных кредитов в зависимости от уровня дохода:

In [35]:
def income_debt(row):
    income = row['total_income_category']
    debt = row['debt']
    if debt ==1:
        if income == 'A':
            return 'A'
        if income == 'B':
            return 'B'
        if income == 'C':
            return 'C'
        if income == 'D':
            return 'D'
        if income == 'E':
            return 'E'
df['income_debt'] = df.apply(income_debt, axis=1)
print(df['income_debt'].value_counts()) 
print(df['total_income_category'].value_counts())

C    1360
B     356
D      21
A       2
E       2
Name: income_debt, dtype: int64
C    16016
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64


Сведем итоговый датафрейм и посчитаем долю просроченных кредитов по категориям.

In [36]:


# через pivot
table_income=df.pivot_table(index='total_income_category', values='debt',aggfunc=['count','sum'])
table_income['debt share']=table_income['sum']/table_income['count']
table_income.columns=['всего взяли кредит','из них просрочено', 'доля просроченных']
display(table_income)

,всего взяли кредит,из них просрочено,доля просроченных
total_income_category,,,
A,25,2,0.080000
B,5041,356,0.070621
C,16016,1360,0.084915
D,350,21,0.060000
E,22,2,0.090909


##### Вывод 3:
Наименее отвественными являются заемщики категории С с зарплатой 50001–200000 (8,5% просрочек). Заемщики категории Е хоть и имеют больший процент просрочек, но количество выданных кредитов крайне мало и такая выборка нерепрезентативна. При это доля просроченных кредитов ниже как для более обеспеченной категории клиентов, так и для менее обеспеченной. Таким образом, прямая корреляция между уровнем дохода и просрочкой кредите не прослеживается. Вероятно, заемщики оценивая свое финансовое состяние достаточно точно представляют, какую сумму долга они в состоянии обслуживать. <br> Заёмщики категории С, в свою очередь, находятся в промежуточном положении между "средне-низким" и "средне-высоким" доходом, ввду чего чаще ошибаются в возможности обслужвания долга и допускают просрочки.  

##### Вопрос 4:Как разные цели кредита влияют на его возврат в срок?

Определим количество просроченных кредитов в зависимости от категории целей:

In [37]:
def purpose_debt(row):
    purpose = row['purpose_category']
    debt = row['debt']
    if debt ==1:
        if purpose == 'операции с недвижимостью':
            return 'операции с недвижимостью'
        if purpose == 'операции с автомобилем':
            return 'операции с автомобилем'
        if purpose == 'получение образования':
            return 'получение образования'
        if purpose == 'проведение свадьбы':
            return 'проведение свадьбы'
df['purpose_debt'] = df.apply(purpose_debt, axis=1)
print(df['purpose_debt'].value_counts()) 
print(df['purpose_category'].value_counts()) 


операции с недвижимостью    782
операции с автомобилем      403
получение образования       370
проведение свадьбы          186
Name: purpose_debt, dtype: int64
операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64


Сведем итоговый датафрейм и посчитаем долю просроченных кредитов по категориям.

In [38]:

# через pivot
table_purpose=df.pivot_table(index='purpose_category', values='debt',aggfunc=['count','sum'])
table_purpose['debt share']=table_purpose['sum']/table_purpose['count']
table_purpose.columns=['всего взяли кредит','из них просрочено', 'доля просроченных']
display(table_purpose)

,всего взяли кредит,из них просрочено,доля просроченных
purpose_category,,,
операции с автомобилем,4306,403,0.093590
операции с недвижимостью,10811,782,0.072334
получение образования,4013,370,0.092200
проведение свадьбы,2324,186,0.080034


##### Вывод 4:
Наиболее ответственными кредитопользователями являются заемщики, взявшие кредит на операции с недвижимостью (7,2% просрочек), операции с автомобилем и кредит на получение образования наиболее часто подвержены просрочкам (9,4% и 9,2% просрочек, соответственно).   

## Общий вывод:

Надежность заемщиков продиктована множеством факторов, из которых сложно выделить один явно превалирующий. Причем, например, возможность обслуживания долга не всегда коррелирует с уровнем дохода. 
<br>Также, кредитоспособность в целом отвественной категории женатых людей сильно зависит от наличия и количества детей в семье.
<br>В части целей кредита наиболее безопасными являются кредиты на недвижимость - нежелание лишиться жилья делает такие кредиты для заёмщика приоритетными. И заемщик готов допускать просрочки по другим кредитам, но не по жилищному. А также кредиты на проведение свадьбы, т.к. такие займы обычно невелики и предполагают быстрое погашение. Кредиты на получение образования имеют частые просрочки из-за невозможности быстро найти работу и стабильно отдавать кредит,  а также того факта, что "образование" у заемщика уже не "изымут". Автокредиты имеют частые просрочки ввиду того, что автомобиль не является предметом первой необходимости, и в случае сложного финансового положения заемщика может быть
возвращён банку.